In [ ]:
from gait3d.gaitanalysis_with_top_cam import RecordingTop

from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
input_dir = Path('/mnt/c/Users/dsege/Downloads/DLC_data/DLC_data/')
week_id = 1
verbose = True
show_plots = True
out_dir_path = Path('/mnt/c/Users/dsege/Downloads/DLC_data/analyzed_files/')

In [ ]:
files = [file for file in input_dir.iterdir() if file.name.endswith('filtered.csv')] # change to not end with filtered!
for file in files:
    recording = RecordingTop(filepath = Path(file), week_id = week_id)
    recording.preprocess()
    if recording.logs['coverage_critical_markers'] >= recording.logs['coverage_threshold']: 
        recording.run_behavioral_analyses()
        recording.export_results(out_dir_path = out_dir_path)
        if verbose:
            print(f'Done with analysis of file: {file.name}:')
            print(f'... coverage of "CenterOfGravity" was at: {round(recording.logs["coverage_CenterOfGravity"]*100, 2)}%')
            print(f'... total gait disruption bouts detected: {recording.behavior_df["gait_disruption_bout_id"].unique().shape[0]}')
        if show_plots:
            fig = plt.figure()
            for corner_marker_id in ['MazeCornerClosedRight', 'MazeCornerClosedLeft', 'MazeCornerOpenRight', 'MazeCornerOpenLeft']:
                x, y = recording.logs[f'normalized_{corner_marker_id}_coords']
                plt.scatter(x, y, label = corner_marker_id)
            plt.scatter(recording.bodyparts['CenterOfGravity'].df['x'], recording.bodyparts['CenterOfGravity'].df['y'], alpha = 0.1)
            plt.plot([0, 0, 50, 50, 0], [0, 4, 4, 0, 0], c = 'black')
            plt.show()
    else:
        if verbose:
            print(f'Insufficient tracking coverage for: {file.name}. Continuing with next.')